#### **Train 3 Different Models**

In [0]:
import os
import mlflow
import mlflow.spark

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import (
    LinearRegression,
    DecisionTreeRegressor,
    RandomForestRegressor
)
from pyspark.ml.evaluation import RegressionEvaluator

# Required for Unity Catalog
os.environ["MLFLOW_DFS_TMP"] = "/Volumes/workspace/default/kaggle_volume/mlflow_tmp"

# Load data
data = (
    spark.table("ecommerce.gold.daily_sales")
    .select("total_events", "total_revenue")
    .dropna()
)

train_df, test_df = data.randomSplit([0.8, 0.2], seed=42)

# Feature vector
assembler = VectorAssembler(
    inputCols=["total_events"],
    outputCol="features"
)

train_vec = assembler.transform(train_df).select("features", "total_revenue")
test_vec  = assembler.transform(test_df).select("features", "total_revenue")

# Models
models = {
    "LinearRegression": LinearRegression(labelCol="total_revenue"),
    "DecisionTree": DecisionTreeRegressor(labelCol="total_revenue", maxDepth=5),
    "RandomForest": RandomForestRegressor(labelCol="total_revenue", numTrees=50)
}

evaluator = RegressionEvaluator(
    labelCol="total_revenue",
    predictionCol="prediction",
    metricName="rmse"
)

mlflow.set_experiment("/Shared/mlflow_model_comparison")

for name, model in models.items():
    with mlflow.start_run(run_name=name):
        mlflow.log_param("model", name)
        mlflow.log_param("features", "total_events")

        fitted_model = model.fit(train_vec)
        predictions = fitted_model.transform(test_vec)
        rmse = evaluator.evaluate(predictions)

        mlflow.log_metric("rmse", rmse)
        mlflow.spark.log_model(fitted_model, "model")

        print(f"{name} | RMSE = {rmse:.2f}")


2026/01/21 05:31:32 WARNING mlflow.utils.requirements_utils: Found pyspark version (4.0.0+databricks.connect.17.2.2) contains a local version label (+databricks.connect.17.2.2). MLflow logged a pip requirement for this package as 'pyspark==4.0.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2026/01/21 05:31:36 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/user_tmp_data/spark-0e92ce18-cd41-4903-8246-36/tmpz3i12fis/model, flavor: spark). Fall back to return ['pyspark==4.0.0']. Set logging level to DEBUG to see the full traceback. 
2026/01/21 05:31:36 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


LinearRegression | RMSE = 5108240.78


2026/01/21 05:31:46 WARNING mlflow.utils.requirements_utils: Found pyspark version (4.0.0+databricks.connect.17.2.2) contains a local version label (+databricks.connect.17.2.2). MLflow logged a pip requirement for this package as 'pyspark==4.0.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2026/01/21 05:31:48 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/user_tmp_data/spark-0e92ce18-cd41-4903-8246-36/tmpm6bt4peu/model, flavor: spark). Fall back to return ['pyspark==4.0.0']. Set logging level to DEBUG to see the full traceback. 
2026/01/21 05:31:48 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


DecisionTree | RMSE = 5832868.43


2026/01/21 05:31:59 WARNING mlflow.utils.requirements_utils: Found pyspark version (4.0.0+databricks.connect.17.2.2) contains a local version label (+databricks.connect.17.2.2). MLflow logged a pip requirement for this package as 'pyspark==4.0.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2026/01/21 05:32:02 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/user_tmp_data/spark-0e92ce18-cd41-4903-8246-36/tmpid0ee0_d/model, flavor: spark). Fall back to return ['pyspark==4.0.0']. Set logging level to DEBUG to see the full traceback. 
2026/01/21 05:32:02 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


RandomForest | RMSE = 5680350.66


#### **Compare Metrics in MLflow**

In [0]:
import mlflow

runs = mlflow.search_runs()

runs[[
    "tags.mlflow.runName",
    "metrics.rmse",
    "params.model"
]]


,tags.mlflow.runName,metrics.rmse,params.model
0,RandomForest,5.680351e+06,RandomForest
1,DecisionTree,5.832868e+06,DecisionTree
2,LinearRegression,5.108241e+06,LinearRegression
3,linear_model,5.680351e+06,LinearRegression


In [0]:
clean_runs = runs[[
    "tags.mlflow.runName",
    "metrics.rmse",
    "params.model"
]].rename(columns={
    "tags.mlflow.runName": "run_name",
    "metrics.rmse": "rmse",
    "params.model": "model"
})

clean_runs


,run_name,rmse,model
0,RandomForest,5.680351e+06,RandomForest
1,DecisionTree,5.832868e+06,DecisionTree
2,LinearRegression,5.108241e+06,LinearRegression
3,linear_model,5.680351e+06,LinearRegression


#### **Build Spark ML Pipeline**

In [0]:
import os
import mlflow
import mlflow.spark

from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

os.environ["MLFLOW_DFS_TMP"] = "/Volumes/workspace/default/kaggle_volume/mlflow_tmp"

data = (
    spark.table("ecommerce.gold.daily_sales")
    .select("total_events", "total_revenue")
    .dropna()
)

train, test = data.randomSplit([0.8, 0.2], seed=42)

assembler = VectorAssembler(
    inputCols=["total_events"],
    outputCol="features"
)

rf = RandomForestRegressor(
    labelCol="total_revenue",
    numTrees=50
)

pipeline = Pipeline(stages=[assembler, rf])

with mlflow.start_run(run_name="RandomForest_Pipeline"):
    pipeline_model = pipeline.fit(train)
    predictions = pipeline_model.transform(test)

    evaluator = RegressionEvaluator(
        labelCol="total_revenue",
        predictionCol="prediction",
        metricName="rmse"
    )

    rmse = evaluator.evaluate(predictions)

    mlflow.log_param("model", "RandomForestPipeline")
    mlflow.log_param("features", "total_events")
    mlflow.log_metric("rmse", rmse)
    mlflow.spark.log_model(pipeline_model, "pipeline_model")

    print("Pipeline RMSE:", rmse)


2026/01/21 05:34:19 WARNING mlflow.utils.requirements_utils: Found pyspark version (4.0.0+databricks.connect.17.2.2) contains a local version label (+databricks.connect.17.2.2). MLflow logged a pip requirement for this package as 'pyspark==4.0.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2026/01/21 05:34:22 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/user_tmp_data/spark-0e92ce18-cd41-4903-8246-36/tmpcf4qssjy/model, flavor: spark). Fall back to return ['pyspark==4.0.0']. Set logging level to DEBUG to see the full traceback. 
2026/01/21 05:34:22 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Pipeline RMSE: 5680350.65519515


#### **Select Best Model**

In [0]:
best_run = clean_runs.sort_values("rmse").iloc[0]

print("Best Model:", best_run["model"])
print("Best RMSE:", best_run["rmse"])


Best Model: LinearRegression
Best RMSE: 5108240.7764305165
